In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import os
import requests
import random
import google
from googlesearch import search
import re

# Get List of Songs

In [2]:
TOKEN = 'BQCxZosrh8zeonGgkOh-Vksizvd8J8PzWXo22r3ahqyu6VmEzJssCgQVV5E0B0j5TO8nSzrxbg_Kt8yPZpBalBG-G35w6G0EdVQZaR_tAstsrUfJu9caK1CEP2VdxZ-pPQsQnTzwRORbTTOdh4e1kew6smFfcQ'

In [3]:
# base URL of all Spotify API endpoints
headers = {
    'Authorization': 'Bearer {token}'.format(token=TOKEN)
}

BASE_URL = 'https://api.spotify.com/v1/playlists/{}/tracks?offset={}'

# PLAYLIST_URL = ['https://open.spotify.com/playlist/38uFgdleiwHcs0tVmTwkdM?si=973358985c54467b', 'https://open.spotify.com/playlist/5Wlpb171qbV5BQ3KSeSku1?si=406b18056b3345fd']
# PLAYLIST_URI = list(map(lambda x:x.split('/')[4].split('?')[0],PLAYLIST_URL))

# final_result = []

# for URI in PLAYLIST_URI:
#     for offset in range(0, 1500, 100):
#         try:
#             r = requests.get(BASE_URL.format(URI, offset), headers=headers)
#             results = r.json()['items']
#             final_result += (list(map(lambda x : {"name": x['track']['name'].split('-')[0].split('(')[0], "artist": x['track']['artists'][0]['name']} , results)))
#             print('Results now have {} songs.'.format(len(final_result)))
#         except Exception as e:
#             print(f'Error: {e}')



In [5]:
# print(f'We now have a list of {len(final_result)} songs.')

In [14]:
# base URL of all Spotify API endpoints
def getSongList(token, playlist_url):
    headers = {
        'Authorization': 'Bearer {}'.format(token)
    }

    BASE_URL = 'https://api.spotify.com/v1/playlists/{}/tracks?offset={}'
    
    PLAYLIST_URI = playlist_url.split('/')[4].split('?')[0]

    final_result = []
    
    totalSongs = int(requests.get(BASE_URL.format(PLAYLIST_URI, '0'), headers=headers).json()['total'])

    for offset in range(0, (totalSongs//100+1) * 100, 100):
        try:
            r = requests.get(BASE_URL.format(PLAYLIST_URI, offset), headers=headers)
            results = r.json()['items']
            final_result += (list(map(lambda x : {"name": x['track']['name'].split('-')[0].split('(')[0], "artist": x['track']['artists'][0]['name']} , results)))
            
        except Exception as e:
            print(f'Error: {e}')
    
    print('{} songs were added from {}.'.format(len(final_result), playlist_url))
    return final_result

# Search Google for Link (Might Get Banned)


In [15]:
# def getSongLink(song):
#     query = '{} by {} lyrics'.format(song['name'], song['artist'])
#     search_results = []
#     for j in search(query, tld="co.in", num=10, stop=10, pause=2):
#         if 'kkbox' in j:
#             return j
#     for j in search(query, tld="co.in", num=10, stop=10, pause=2):
#         if 'mojim' in j:
#             return j
    
#     return None
    

In [16]:
# random_song = final_result[random.randint(0,len(final_result))]
# query = '{} by {} lyrics'.format(random_song['name'], random_song['artist'])
# print(query)
# print(getSongLink(random_song))

# Scrape KKBox for Link (free)

In [17]:
def getSongLinkFromKkbox(song):
    url = 'https://www.kkbox.com/hk/en/search.php?word={}'.format(song['name'])
    try:
        r = requests.get(url)
        pageContent = r.content 
    except Exception as e:
        print('Error requesting top results link {}'.format(e))

    soup = BeautifulSoup(r.content, 'html5lib')   
    try:
        link = 'https://www.kkbox.com/' + str(soup.find('a', attrs = {'class':'song-title'})).split('href=')[1].split('"')[1]
        return link
    except Exception as e:
        print('{} has no link, passing.'.format(song['name']))
        return None
    
# random_song = final_result[random.randint(0,len(final_result))]
# query = '{} by {} lyrics'.format(random_song['name'], random_song['artist'])
# print(query)
# print(getSongLinkFromKkbox(random_song))

# Scrape Lyrics Out of Web

In [18]:
def rowFilter(row):
    if '演唱' in row:
        return False
    if '作詞' in row:
        return False
    if '作曲' in row:
        return False
    if 'Lyricist' in row:
        return False
    if 'Composer'  in row:
        return False
    if '間奏' in row:
        return False
    if '編' in row:
        return False
    if '監' in row:
        return False
    if '曲' in row:
        return False
    
    return True

def replaceKeywords(lyrics):
    wordsToReplace = {" " : "", 
                      "</>" : "", 
                      "<br/>" : "\n",
                      "<p>": "",
                      "</p>": "",
                      '<divclass="lyrics">': "",
                      '</div>' : "",
                      '(女)': '',
                      '(男)': '',
                      '(合)': '',
                      '#': "",
                      '＃': '',
                      '(': '',
                      ')': '',
                      ',':'\t',
                      '’': '',
                      '!': '',
                      '…': '',
                      '＊': '',
                      '*': '',
                      "（": '',
                      "'":"",
                      '...':'',
                      '☆':'',
                      '詞：':'',
                      ':': '',
                      '/':''
                     }
    result = lyrics
    for word in wordsToReplace.keys():
        result = result.replace(word, wordsToReplace[word])
    
    return result
    
    
def scrapeKkboxLink(link):
    try:
        r = requests.get(link)
        pageContent = r.content
    except Exception as e:
        print('Error requesting link during scraping {}'.format(e))
        
    soup = BeautifulSoup(r.content, 'html5lib')   
    lyrics = replaceKeywords(str(soup.find('div', attrs = {'class':'lyrics'})))
    lyrics = lyrics.split('\n')
    lyrics = [row for row in lyrics if rowFilter(row)]
    lyrics = '\n'.join(lyrics)
    lyrics = re.sub(r'[a-zA-z]', '', lyrics).replace('+','')

    return lyrics

def scrapeMojimLink(link):
    try:
        r = requests.get(link)
        pageContent = r.content
    except Exception as e:
        print('Error requesting link during scraping {}'.format(e))
        
    soup = BeautifulSoup(r.content, 'html5lib')   
    lyrics = str(soup.find('dd', attrs = {'class':'fsZx3'})).replace("<br/>", "\n").replace('<dd class="fsZx3" id="fsZx3">','').replace('</dd>','').replace('更多更詳盡歌詞 在 <a href="http://mojim.com">※ Mojim.com　魔鏡歌詞網 </a>','')
    lyrics = lyrics.split('\n')
    lyrics = [row for row in lyrics if '<ol>' not in row]
    lyrics = '\n'.join(lyrics)
    return lyrics

def getLyrics(link):
    if link == None:
        return None
    if 'kkbox' in link:
        results = scrapeKkboxLink(link)
        if len(results.replace('\n','')) < 100:
            return None
        
        return results
    if 'mojim' in link:
        return scrapeMojimLink(link)
    
    return None

In [19]:
# print(getLyrics(getSongLinkFromKkbox(random_song)))

# Summarising what the flow is:

- 1) Get all songs from spotify playlist
- 2) For each song,
    - a) Search KKBox for top results
   -  b) Return top result link for lyrics
   - c) Go to that link and scrape the lyrics
    - d) Flow lyrics through and save it for the song

In [20]:
# Create an sqlite3 to store all the scraped lyrics
import sqlite3
import time

con = sqlite3.connect('ai_love_song.db')
cur = con.cursor()
sql_statements = []

In [21]:
def createDB():
    cur.execute('''CREATE TABLE IF NOT EXISTS lyrics (run_id text, name text, artist text, lyrics text)''')
    con.commit()

def addStatement(sql):
    global sql_statements
    if len(sql_statements) > 50:
        return pushTransaction()
    
    sql_statements.append(sql)

def pushTransaction():
    global sql_statements
    rows_added = 0
    rows_error = 0
    cur.execute('BEGIN TRANSACTION')
    for statement in sql_statements:
        try:
            cur.execute(statement)
            rows_added += 1
        except Exception as e:
            print('Row cannot be added because: ', e)
            rows_error += 1
            pass
    sql_statements = []
    con.commit()
    print("Transaction Completed: {} rows added ; {} rows error".format(rows_added, rows_error))
    
def insertSong(runId, song):
    songUrl = getSongLinkFromKkbox(song)
    lyrics = getLyrics(songUrl)
    if lyrics == None:
        return False
    sql = '''INSERT INTO lyrics VALUES("{}", "{}", "{}", "{}") '''.format(runId, song['name'], song['artist'], lyrics)
    addStatement(sql)
    return True

def deleteRun(runId):
    sql = '''DELETE FROM lyrics WHERE run_id={}'''.format(runId)
    cur.execute(sql)
    con.commit()

In [22]:
apiToken= 'BQAqO8O4XzJNR3WN8Riiy4ufJPt4CkINoLuJducu81QRs-U3pOyDG0F7026KlZqIhSmZLgev76FcXRDEIg0xl048oB_5SggG06M-HfziRyTfGH3yRkZo9c8kRlP7sfJee9SsG1qBgIvbFgZvU0OzhZo8Lcln1g'
test = getSongList(apiToken, 'https://open.spotify.com/playlist/38uFgdleiwHcs0tVmTwkdM?si=035f80a093fb41a6')

1085 songs were added from https://open.spotify.com/playlist/38uFgdleiwHcs0tVmTwkdM?si=035f80a093fb41a6.


In [23]:
# Create the databse first if not exist
createDB()

playlists = [
    'https://open.spotify.com/playlist/4cd77a6cpcSyJJADQm3CaD?si=7fd714df40584946',
    'https://open.spotify.com/playlist/0xIfZKAKffK7Xw8kHl0uO1?si=690bbb31d288463a',
    'https://open.spotify.com/playlist/4YGjtvMmG7ePQr2SZt8U3I?si=9a64aa9320a248f7'
]


song_list = []
for playlist in playlists:
    song_list += getSongList(apiToken, playlist)

print('Song list now contains {} songs'.format(len(song_list)))

463 songs were added from https://open.spotify.com/playlist/4cd77a6cpcSyJJADQm3CaD?si=7fd714df40584946.
773 songs were added from https://open.spotify.com/playlist/0xIfZKAKffK7Xw8kHl0uO1?si=690bbb31d288463a.
1333 songs were added from https://open.spotify.com/playlist/4YGjtvMmG7ePQr2SZt8U3I?si=9a64aa9320a248f7.
Song list now contains 2569 songs


In [17]:
# deleteRun('1')

In [28]:
sql_statements = []

for count, song in enumerate(song_list):

    print('{} : Inserting {} by {}.....................'.format(count ,song['name'], song['artist']), end='\r')
    insertSong('4', song)
    time.sleep(0.5)

pushTransaction()
        
    

Transaction Completed: 51 rows added ; 0 rows error........
Transaction Completed: 51 rows added ; 0 rows error...........
Transaction Completed: 51 rows added ; 0 rows error...............
Transaction Completed: 51 rows added ; 0 rows error.............
Transaction Completed: 51 rows added ; 0 rows error......ng.....................
Row cannot be added because:  near "我愛你": syntax error......
Row cannot be added because:  near "真愛總會是永遠": syntax error
Transaction Completed: 49 rows added ; 2 rows error
Transaction Completed: 51 rows added ; 0 rows error...........
Transaction Completed: 51 rows added ; 0 rows error..............
Transaction Completed: 51 rows added ; 0 rows error.............
Transaction Completed: 51 rows added ; 0 rows error....................
Transaction Completed: 51 rows added ; 0 rows error.............
 has no link, passing.uis Koo..............................
Transaction Completed: 51 rows added ; 0 rows error.......
Transaction Completed: 51 rows added ; 0 r

UnboundLocalError: local variable 'r' referenced before assignment

In [33]:
index = 0

for s in song_list:
    if '請你不企近黃線' in s:
        break
    else:
        index+=1

print(index)

2569
